In [5]:
import os
import time
import json
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# ✅ .env에서 API 키 불러오기
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ 설정
topics = ["음식", "날씨", "취미"]
samples_per_topic = 20  # 예시 수 적게 설정 후 검증 (실행 전 1000 등으로 확장 가능)

# ✅ GPT 호출 함수
def generate_dialog(topic):
    prompt = f"""
'{topic}'에 대해 자연스럽고 일상적인 3턴 대화를 만들어줘. 
형식:
질문: ...
답변: ...
역질문: ...
맥락 흐름이 어색하지 않도록 자연스럽게 연결해줘.
"""
    try:
        chat_completion = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {"role": "system", "content": "너는 한국어 일상 대화에 능숙한 챗봇이야."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8,
            max_tokens=300
        )
        content = chat_completion.choices[0].message.content
        lines = [line.strip() for line in content.split("\n") if line.strip()]
        if len(lines) == 3:
            q = lines[0].replace("질문:", "").strip()
            a = lines[1].replace("답변:", "").strip()
            f = lines[2].replace("역질문:", "").strip()
            if len(q) > 5 and len(a) > 5 and len(f) > 5:
                return {"주제": topic, "질문": q, "답변": a, "역질문": f}
    except Exception as e:
        print("⚠️ 오류 발생:", e)
    return None

# ✅ 생성 루프
all_rows = []
for topic in topics:
    print(f"🔹 '{topic}' 생성 중...")
    count = 0
    tries = 0
    while count < samples_per_topic and tries < samples_per_topic * 2:
        row = generate_dialog(topic)
        tries += 1
        if row:
            all_rows.append(row)
            count += 1
            print(f"✔️ {topic} {count}/{samples_per_topic}")
        time.sleep(1.1)  # rate limit 대비

# ✅ 데이터 저장
df = pd.DataFrame(all_rows).drop_duplicates(subset=["질문", "답변", "역질문"])
df.to_csv("gpt_smalltalk_ko.csv", index=False, encoding="utf-8-sig")
with open("gpt_smalltalk_ko.jsonl", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        f.write(json.dumps(row.to_dict(), ensure_ascii=False) + "\n")

print(f"✅ 생성 완료! 총 {len(df)}개 저장됨")


🔹 '음식' 생성 중...
✔️ 음식 1/20
✔️ 음식 2/20
✔️ 음식 3/20
✔️ 음식 4/20
✔️ 음식 5/20
✔️ 음식 6/20
✔️ 음식 7/20
✔️ 음식 8/20
✔️ 음식 9/20
✔️ 음식 10/20
✔️ 음식 11/20
✔️ 음식 12/20
✔️ 음식 13/20
✔️ 음식 14/20
✔️ 음식 15/20
✔️ 음식 16/20
✔️ 음식 17/20
✔️ 음식 18/20
✔️ 음식 19/20
✔️ 음식 20/20
🔹 '날씨' 생성 중...
✔️ 날씨 1/20
✔️ 날씨 2/20
✔️ 날씨 3/20
✔️ 날씨 4/20
✔️ 날씨 5/20
✔️ 날씨 6/20
✔️ 날씨 7/20
✔️ 날씨 8/20
✔️ 날씨 9/20
✔️ 날씨 10/20
✔️ 날씨 11/20
✔️ 날씨 12/20
✔️ 날씨 13/20
✔️ 날씨 14/20
✔️ 날씨 15/20
✔️ 날씨 16/20
✔️ 날씨 17/20
✔️ 날씨 18/20
✔️ 날씨 19/20
✔️ 날씨 20/20
🔹 '취미' 생성 중...
✔️ 취미 1/20
✔️ 취미 2/20
✔️ 취미 3/20
✔️ 취미 4/20
✔️ 취미 5/20
✔️ 취미 6/20
✔️ 취미 7/20
✔️ 취미 8/20
✔️ 취미 9/20
✔️ 취미 10/20
✔️ 취미 11/20
✔️ 취미 12/20
✔️ 취미 13/20
✔️ 취미 14/20
✔️ 취미 15/20
✔️ 취미 16/20
✔️ 취미 17/20
✔️ 취미 18/20
✔️ 취미 19/20
✔️ 취미 20/20
✅ 생성 완료! 총 60개 저장됨


In [3]:
%pip install pandas asyncio dotenv openai tqdm datetime

%pip install --upgrade pip

%pip install --upgrade pandas

%pip install --upgrade asyncio

%pip install --upgrade dotenv

%pip install --upgrade openai

%pip install --upgrade tqdm

%pip install --upgrade datetime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import pandas as pd
import nest_asyncio
import asyncio
from dotenv import load_dotenv
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm_asyncio
from datetime import datetime

nest_asyncio.apply()
load_dotenv()
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

TOPICS = {
    "food": "음식에 관한 친근한 스몰토크",
    "weather": "날씨에 대한 자연스러운 대화",
    "hobby": "취미 관련 이야기를",
}

def build_conversation_prompt(topic_kr, n_turns=10):
    return f"""\
{topic_kr}를 주제로 자연스럽고 편안한 {n_turns}턴짜리 대화를 만들어줘.

조건:
- 질문 → 답변 → 다시 묻기 방식
- "A:", "B:" 로 시작하는 라인 형식
- 말투는 일상적이고 자연스럽게
- 중복된 표현 피하고 다양한 표현 사용
"""

async def generate_turns(topic_key, topic_kr, n_samples):
    unique_conversations = set()
    for _ in tqdm_asyncio(range(n_samples), desc=f"[{topic_key}] 생성 중"):
        try:
            prompt = build_conversation_prompt(topic_kr)
            response = await client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.85,
                max_tokens=1024,
            )
            result = response.choices[0].message.content.strip()
            unique_conversations.add(result)
        except Exception as e:
            print(f"[{topic_key}] 오류 발생: {e}")
    return list(unique_conversations)

async def main(total_per_topic=5000):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    for topic_key, topic_kr in TOPICS.items():
        print(f"\n🔹 '{topic_key}' 주제 생성 시작...")
        results = await generate_turns(topic_key, topic_kr, total_per_topic)
        df = pd.DataFrame({"topic": topic_key, "conversation": results})
        filename = f"{topic_key}_chats_{timestamp}.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"✅ 저장 완료: {filename} (총 {len(df)}개)")

# ✅ Jupyter에서는 이렇게 실행
await main(total_per_topic=5000)



🔹 'food' 주제 생성 시작...


[food] 생성 중:   0%|          | 0/5000 [00:08<?, ?it/s]


CancelledError: 